将文本作为字符串加载到内存中。

将字符串拆分为词元（如单词和字符）。

建立一个词表，将拆分的词元映射到数字索引。

将文本转换为数字索引序列，方便模型操作。

In [1]:
import collections
import re
from d2l import torch as d2l


In [2]:
# 读取一本书
d2l.DATA_HUB["time_machine"] = (d2l.DATA_URL+"timemachine.txt","090b5e7e70c295757f55df93cb0a180b9691891a")

In [3]:
def read_time_machine():
    with open(d2l.download("time_machine"),"r") as f:
        lines = f.readlines()
    return [re.sub("[^A-Za-z]+"," ",line).strip().lower() for line in lines]

lines = read_time_machine()

In [4]:
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [5]:
# 词元化 或者是 token化
def tokennize(lines,token="word"):
    if token=="word":
        return [line.split() for line in lines]
    elif token=="char":
        return [list(line) for line in lines]
    else:
        print("错误")

In [6]:
tokens = tokennize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [7]:
def count_corpus(tokens):
    if len(tokens)==0 or isinstance(tokens[0],list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [14]:
## 构建词表
class Vocab:
    
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens==None:
            tokens=[]
        if reserved_tokens==None:
            reserved_tokens=[]
        
        # 按出现的频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(),key=lambda x:x[1],reverse=True)
        # 未知词元的索引是0
        self.idx_to_token = ["<unk>"]+reserved_tokens
        self.token_to_index = {token:idx for idx,token in enumerate(self.idx_to_token)}

        for token,freq in self._token_freqs:
            if freq<min_freq:
                break
            if token not in self.token_to_index:
                self.idx_to_token.append(token)
                self.token_to_index[token] = len(self.idx_to_token)-1

    def __len__(self):
        return len(self.idx_to_token)
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_index.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [15]:
vocab = Vocab(tokens)
print(list(vocab.token_to_index.items())[:10])
print(list(vocab._token_freqs)[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]
[('the', 2261), ('i', 1267), ('and', 1245), ('of', 1155), ('a', 816), ('to', 695), ('was', 552), ('in', 541), ('that', 443), ('my', 440)]


In [16]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [17]:
print(len(vocab))

4580


In [18]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokennize(lines,"char")
    vocab = Vocab(tokens)
    
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)